# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_erratic,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.8554482108333332 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOUSEHOLD_1_173_CA_1_validation,3.0,5.520226,0.1,0.1
1,HOUSEHOLD_1_173_CA_1_validation,16.0,5.290949,0.1,0.1
2,HOUSEHOLD_1_173_CA_1_validation,8.0,6.292266,0.1,0.1
3,HOUSEHOLD_1_173_CA_1_validation,2.0,6.455418,0.1,0.1
4,HOUSEHOLD_1_173_CA_1_validation,1.0,6.036606,0.1,0.1
...,...,...,...,...,...
119875,HOUSEHOLD_1_151_CA_4_validation,2.0,4.967427,0.9,0.9
119876,HOUSEHOLD_1_151_CA_4_validation,7.0,2.328079,0.9,0.9
119877,HOUSEHOLD_1_151_CA_4_validation,1.0,6.528662,0.9,0.9
119878,HOUSEHOLD_1_151_CA_4_validation,4.0,1.553360,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.1361076948833348 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3235,HOUSEHOLD_2_499_CA_2_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3236,HOUSEHOLD_2_499_CA_2_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3237,HOUSEHOLD_2_499_CA_2_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3238,HOUSEHOLD_2_499_CA_2_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3235,HOUSEHOLD_2_499_CA_2_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3236,HOUSEHOLD_2_499_CA_2_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3237,HOUSEHOLD_2_499_CA_2_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3238,HOUSEHOLD_2_499_CA_2_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3235,HOUSEHOLD_2_499_CA_2_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3236,HOUSEHOLD_2_499_CA_2_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3237,HOUSEHOLD_2_499_CA_2_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3238,HOUSEHOLD_2_499_CA_2_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3235,HOUSEHOLD_2_499_CA_2_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3236,HOUSEHOLD_2_499_CA_2_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3237,HOUSEHOLD_2_499_CA_2_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3238,HOUSEHOLD_2_499_CA_2_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3235,HOUSEHOLD_2_499_CA_2_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
3236,HOUSEHOLD_2_499_CA_2_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
3237,HOUSEHOLD_2_499_CA_2_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
3238,HOUSEHOLD_2_499_CA_2_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 17.5 %
Percentage of unexpected values of WMAPE is: 17.5 %
Percentage of unexpected values of SMAPE is: 20.0 %
Percentage of unexpected values of MAPE is: 17.5 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE      SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                               
0.1   0.1   0.834073  66.859956  76.177994  71.103991  3.541585  4.641345
      0.2   0.832001  66.654044  76.698387  72.384862  3.547663  4.653268
      0.3   0.832254  66.683219  77.317387  73.498316  3.560372  4.673650
      0.4   0.834731  66.922275  78.120800  74.692763  3.577726  4.695189
      0.5   0.838840  67.300994  79.123006  75.950294  3.597480  4.717345
...              ...        ...        ...        ...       ...       ...
0.9   0.5   0.937112  75.707363  86.921822  82.636034  4.013321  5.262968
      0.6   0.941069  76.050831  87.774441  83.677787  4.032477  5.296063
      0.7   0.946989  76.578519  89.057525  85.026819  4.055243  5.330546
      0.8   0.953949  77.207791  90.692447  86.506753  4.079359  5.367495
      0.9   0.963387  78.044441  93.234684  88.546134  4.110409  5.408536

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.2, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE     SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                            
0.1   0.1   0.825709  65.01075  72.92980  71.58560  2.775820  3.393371
      0.2   0.821605  65.61755  72.59030  71.26140  2.732592  3.323982
      0.3   0.820919  67.35625  73.22200  71.66430  2.690950  3.293278
      0.4   0.820545  67.97010  74.93765  72.74580  2.559169  3.281461
      0.5   0.820095  67.88150  76.66390  74.65235  2.559781  3.280162
...              ...       ...       ...       ...       ...       ...
0.9   0.5   0.932783  73.34900  84.26205  82.00695  3.080748  3.942889
      0.6   0.932783  74.70950  85.00780  82.80970  3.103885  3.987395
      0.7   0.937064  75.48430  87.52335  83.55055  3.071506  4.038133
      0.8   0.947396  75.80375  90.51265  85.66405  3.038605  4.075992
      0.9   0.956033  76.11915  94.36255  88.70095  3.014364  4.076832

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.2, 0.3)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of MAPE is: (0.2, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.4)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.5)


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.1
best_beta = 0.1

In [ ]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.00913912458333357 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1,HOUSEHOLD_1_106_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
2,HOUSEHOLD_1_118_CA_2_validation,1.338394,111.8287,130.1879,124.5674,4.632903,5.492857
3,HOUSEHOLD_1_147_CA_2_validation,0.743053,44.4561,51.1924,48.3952,1.428948,1.679231
4,HOUSEHOLD_1_151_CA_2_validation,0.722580,60.7940,63.8904,76.7889,2.779153,3.440104
5,HOUSEHOLD_1_151_CA_4_validation,0.656740,63.4719,64.2927,75.8406,1.768147,2.487534
6,HOUSEHOLD_1_173_CA_1_validation,0.749515,56.8009,66.1702,86.5474,3.286336,5.442608
7,HOUSEHOLD_1_173_CA_2_validation,0.984478,80.9105,101.2395,42.2062,2.196143,2.706926
8,HOUSEHOLD_1_179_CA_1_validation,0.686553,60.0734,71.0767,39.6823,1.373105,1.670732
9,HOUSEHOLD_1_204_CA_1_validation,0.831354,82.2219,83.2792,114.7698,2.877765,4.847396


In [ ]:
df_result_final.to_csv('CrostonTSB_Erratic_Test_Data.csv')